# Google Drive connection

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pwd

Mounted at /content/drive
/content


In [2]:
import random
import pandas as pd
import numpy as np
import time
import math


# Ratio
---

In [ ]:
class RATIO():
  def __init__(self, Ratio_List: list = []):
    self.Ratio_List = Ratio_List
    self.seed=47

    sheet_id = ""
    sheet_name = "sheet1"
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    df = pd.read_csv(url)
    self.Asset_Names = list(df.columns)

    self.integrated_dict = []

    self.stock_list = []
    '''자산별 인덱스 저장'''
    self.share_list = []
    '''자산별 인덱스당 몇 주,
    자산별 인덱스랑 내부 인덱스 위치 맞춰서'''
    self.price_list = []
    '''자산별 투입 비용 기록,
    현재 가치는 "몇주 X 현재가" 가 결정'''

    for i in range(len(self.Asset_Names)):
      self.stock_list.append([])
      self.share_list.append([])
      self.price_list.append([])



  def Get_Asset_Names(self):
    return self.Asset_Names



  def in_data(self, Data_List: list = [], Data_Name: list = [], Name_List: list = []):
    '''
    agent마다 다른 종목 아님 자산 이런식으로 한다면, 자산 비율 봐서 데이터 미리 안넣어줄수도

    ### examples
    ins = []
    for i in range(자산개수):
      ins.append([])
      for j in range(각 자산 내 종목 수):
        ins[i].append(종목[j])
    agent.in_data(ins, agent.Get_Asset_names())
    '''
    self.Data_List = Data_List
    '''[[자산종류1...], [2], [3]]'''
    self.Name_List = Name_List
    self.Data_Name = Data_Name
    '''[종류1, 종류2...]'''



  def softmax(self, a: list = []):
    a = np.array(a)
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a/sum_exp_a
    return y.tolist()


  def list_return(self, i: int):
    return self.Asset_Names[i], self.stock_list[i], self.share_list[i], self.price_list[i]




# Money

In [ ]:
class MONEY():
  def __init__(self, percentile: int = 1):
    sheet_id = ""
    sheet_name = "sheet1"
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    self.money_df = pd.read_csv(url)[["Whole", "Salary_2019", "Salary_2024"]].iloc[percentile-1]

    url = ''
    self.track_df = pd.read_csv(url)  # 판매한 거랑 손익만 따지면 로그로 충분 ?

    self.whole_input = self.money_df["Whole"] # 월급날마다 더하기만 하고 마지막에 비교

  def salary(self, date: time.strftime("YY-mm-dd")):
    exp_year = (int(date.year) - 2019)/6
    exp_month = (int(date.month)-1)/72
    return self.money_df["Salary_2019"]*((1-exp_year)-exp_month) + self.money_df["Salary_2024"]*(exp_year+exp_month)


# Agent

In [43]:
class AGENT(RATIO, MONEY):
  def __init__(self, Ratio_List: list = [], percentile: int = 1):
    RATIO.__init__(self, Ratio_List)
    MONEY.__init__(self, percentile)
    self.this_month = None



  def payday(self, date: time.strftime("YY-mm-dd")):
    if self.this_month == None:
      self.this_month = date.month

    if self.this_month == date.month:
      return
    else:
      self.this_month = date.month

    # 월급날
    self.money_df["Whole"] = self.money_df["Whole"] + self.salary(date)
    self.whole_input += self.salary(date)


  def currency(self, mode: int = 0, money: int = 0):
    if mode == 0:
      return

    elif mode == 1:
      return



  def buy_each(self, i: int, Index: int, money: int, date: time.strftime("YY-mm-dd")):
    '''
    i: 자산 종류 index
    Index: 자산 종류 내 종목 인덱스
    share: 몇 주
    date: 시뮬레이션 턴
    '''
    stock_list = self.stock_list[i]
    share_list = self.share_list[i]
    price_list = self.price_list[i]
    Data_List  = self.Data_List[i]

    # ratio 0 -> no buy
    if self.Ratio_List[i] == 0:
      return


    # 현재 주식이 상장은 했는가  /  Data_List[i][j].index[0].year
    date_list = list(Data_List[Index].index)
    if date_list == []:  # 애초에 241231까지 비상장
      return  # 일단 운없다 돈 세이브
    else:   # 일단 상장은 함
      if date_list[0] > date:  # 에이전트 입력 날짜보다 더 늦게 상장함
        return  # //

    if date not in date_list:
      return


    # 일단 구매 가능한가
    usable_money = money # self.money_df["Whole"] * self.Ratio_List[i]  # money_df 자산 분위는 생성자에서 deter
    howmuch = Data_List[Index].loc[date]["Low"]
    if howmuch == 0: # 상폐 / 거래정지
      return

    # 전체 자산 * 자산 비율 * 각 주 구매 비율 < 최소 1주 가격 -> 못삼 / 미장은 되는데 국장땜에
    if usable_money < howmuch:
      return

    n_share = usable_money // howmuch
    price = n_share * howmuch

    # print(self.Name_List[i][Index])
    # print(Data_List[Index])

    # 자산보유 state 변경
    if Index in stock_list:
      index = self.stock_list[i].index(Index)
      self.share_list[i][index] += n_share
      self.price_list[i][index] += price
    else:  # new
      self.stock_list[i].append(Index)
      self.share_list[i].append(n_share)
      self.price_list[i].append(price)

    # 돈 state 변경
    self.money_df["Whole"] = self.money_df["Whole"] - price




  def sell_each(self, i: int, index: int, share: int, date: time.strftime("YY-mm-dd")):
    '''
    i: 자산 종류 index
    Index: 자산 종류 내 종목 인덱스
    share: 몇 주
    date: 시뮬레이션 턴
    '''
    asset_name = self.Asset_Names[i]
    stock_list = self.stock_list[i]
    share_list = self.share_list[i]
    price_list = self.price_list[i]
    Data_List  = self.Data_List[i]
    Name_List  = self.Name_List[i]
    # print(Name_List[stock_list[index]])

    date_list = list(Data_List[stock_list[index]].index)
    if date not in date_list:  # 현재 날짜 데이터가 없음
      # print('시발자동차')
      # print(date)
      # # print(date_list)
      # print(Data_List[index])
      # print(Name_List[index])
      return

    # 기록 추가
    current_price_byone = Data_List[stock_list[index]]["High"].loc[date]
    if current_price_byone == 0:
      current_price_byone = Data_List[stock_list[index]]["Close"].loc[date]
    if current_price_byone == 0:
      return
    my_price = price_list[index]
    PL = share * current_price_byone - my_price * (share/share_list[index])  # share_list[index]
    stock = Name_List[stock_list[index]]
    # share = share_list[index]
    con = pd.DataFrame([asset_name, stock, share, PL, date]).T
    con.columns = self.track_df.columns
    self.track_df = pd.concat([self.track_df, con])

    # 자산보유 state 변경
    if share_list[index] == share:
      del self.stock_list[i][index]
      del self.share_list[i][index]
      del self.price_list[i][index]
    else:
      self.price_list[i][index] *= 1 - (share/share_list[index])
      self.share_list[i][index] -= share

    # 자산 state 변경
    self.money_df["Whole"] += current_price_byone * share



  def sell_all(self, date: time.strftime("YY-mm-dd")):
    for i in range(len(self.stock_list)):
      turret = 0
      for j in range(len(self.stock_list[i])):
        # _list[i] 안에 계속 삭제되면서 밀림 / 그냥 index=0에 터렛설치
        # try:
        self.sell_each(i, 0, self.share_list[i][turret], date)
        # except:
          # turret+=1


  def result(self, date: time.strftime("YY-mm-dd")):
    '''return 수익률, track_df'''
    self.sell_all(date)

    sum = 0
    # 총 수익률 계산
    for i in range(len(self.track_df)):
      sum += self.track_df["PL"].iloc[i]


    rate_of_return = [sum, self.money_df["Whole"], self.whole_input]  # self.money_df["Whole"] / self.whole_input

    return rate_of_return, self.track_df

# **Agents**
---
### 공통 규칙
- 모든 현금은 항상 **비율에 맞추어 모두** 소진되어야, 5년이면 월급이 더 많음
- 팔면 그 **다음날** 자산에 합쳐 투자
- 매도 매수 전략은 맘대로
- 같은 자산 종류 내에서 최소 **3개**의 다른 종목은 가져야 (채권, 인버스 제외?...알잘딱)
- 추세 지표 쓸때 신호 뜨자마자 사면 맵핵이니까 신호 1-2일 이후 매수/매도
- 시간은 하루 단위, 시간은 시뮬레이션에서 흐름

---
### 구현


> ### RATIO, MONEY 상속
```
class blabla(RATIO, MONEY):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
```
- 그냥 생성자에 이렇게만 하면

---

> ### 시계열 데이터셋 입력 및 저장
```
def in_data():
```
- 시뮬레이션이 던져줄 데이터셋 다룰 메소드, 대충 판다스로 잘
- 자산 종류별로 복수개의 데이터 받을 것

---

> ### 매수
```
def buy():
```
- 내부적으로 현 자산이랑 월급날 이자날 배당일 계산하고 돈 있으면 구매
- 자산별 라벨 리스트에서 랜덤 비율로 라벨 추출, randint(3, 10) 하고 이값으로 이터레이션 돌려서 또 랜덤 리스트 뽑고 소프트맥스
- 현재 보유 자산 기록

---

> ### 매도
```
def sell():
```
- 원금 대비 수익률 기록 / 추적

---

> ### 결과 출력
```
def result()
  return pd.DataFrame()
```
- 시뮬레이션 종료 후 행동 / 수익률 등 데이터 판다스로 반환
- 행동은 이상한 결과 있을때 뭔짓했는지 확인

---
### 고려
- 잡주의 기준은 뭐고 이들을 제외해야하는가
-

> ## *순수 랜덤*

In [44]:
class Pure_Random(AGENT):
  def __init__(self, Ratio_List: list=[], percentile: int=1):
    super().__init__(Ratio_List, percentile)


  def buy(self, date: time.strftime("YY-mm-dd")):
    # 월급날
    self.payday(date)
    whole_money = self.money_df["Whole"]  # 아래에서 자꾸 바뀜

    #* 자산 종류별로
    for i in range(len(self.Asset_Names)):
      # ratio 0 -> no buy
      if self.Ratio_List[i] == 0:
        # print(1)
        continue
      if len(self.Data_List[i]) == 0:
        # print(0)
        continue

      # print('ss')
      howmany = random.randrange(3, 11)  # 얼마나 종목 살건지 랜덤
      to_buy = []   # 살 df ||인덱스||
      buy_ratio = []  # 살 자산 각 상대 비율
      #* 자산 종류 안 /종목 종류
      for j in range(howmany):
        buy_ratio.append(random.random()*4)   # 다이나믹하게 *4 / 안살건 안사게
        to_buy.append(random.randrange(0, len(self.Data_List[i])))   # 다른것들은 최상단(순위권) 선호, 얘는 잡주까지 다
      buy_ratio = self.softmax(buy_ratio)  # norm

      usable_money = whole_money * self.Ratio_List[i]  # money_df 자산 분위는 생성자에서 deter
      for j in range(howmany):
        to_spend_each = usable_money * buy_ratio[j]
        # try:
        self.buy_each(i ,to_buy[j], to_spend_each, date)
        # except:
          # sibal = 0



  def sell(self, date: time.strftime("YY-mm-dd")):
    # prob = random.random()
    # if prob > 0.2:
    for i in range(len(self.Asset_Names)):
      if self.Ratio_List[i] == 0:
        continue
      if len(self.stock_list[i]) == 0:
        continue
      if len(self.Data_List[i]) == 0:
        continue

      for j in range(random.randrange(0, len(self.stock_list[i]))):
        Index = random.randrange(0, len(self.stock_list[i]))
        Share = math.floor(random.random() * self.share_list[i][Index])
        Share = Share if Share != 0 else 1
        # try:
        self.sell_each(i, Index, Share, date)
        # except:
          # sibal = 0


## *존버*
- 매도 구현 안함
- 월급은 다 넣음, 이건 랜덤, 이거도 안팜

In [ ]:
class Zone_Burr(AGENT):
  def __init__(self, Ratio_List: list=[], percentile: int=1):
    super().__init__(Ratio_List, percentile)


  def buy(self, date: time.strftime("YY-mm-dd")):
    # 월급날
    self.payday(date)

    #* 자산 종류별로
    for i in range(len(self.Asset_Names)):
      # ratio 0 -> no buy
      if self.Ratio_List[i] == 0:
        continue
      if len(self.Data_List[i]) == 0:
        # print(0)
        continue

      howmany = random.randrange(3, 11)  # 얼마나 종목 살건지 랜덤
      to_buy = []   # 살 df ||인덱스||
      buy_ratio = []  # 살 자산 각 상대 비율
      #* 자산 종류 안 /종목 종류
      for j in range(howmany):
        buy_ratio.append(random.random()*4)   # 다이나믹하게 *4 / 안살건 안사게
        to_buy.append(random.randrange(0, len(self.Data_List[i])))   # 다른것들은 최상단(순위권) 선호, 얘는 잡주까지 다
      buy_ratio = self.softmax(buy_ratio)  # norm

      usable_money = self.money_df["Whole"] * self.Ratio_List[i]  # money_df 자산 분위는 생성자에서 deter
      for j in range(howmany):
        to_spend_each = usable_money * buy_ratio[j]
        self.buy_each(i ,to_buy[j], to_spend_each, date)



  def sell(self, date: time.strftime("YY-mm-dd")):
    return  # 존-버


### *Buy-The-Dip*

In [ ]:
class Buy_The_Dip():
  def __init__(self):
    self.tries = 0

  def forward(self, x):
    return x

### *MA + RSI*

In [ ]:
# https://alphasquare.co.kr/home/insight/posts/5cf091b8-c40b-4d57-8403-d05ecdb0b044
class f_MA():
  def __init__(self):
    self.tries = 0


In [ ]:
# https://alphasquare.co.kr/home/insight/posts/303a3bf1-a9bf-4d25-a367-3e783e97f2c3
class f_RSI():
  def __init__(self):
    self.tries = 0


In [ ]:
class MA_RSI():
  def __init__(self):
    self.tries = 0


### *Squeeze-Momentum*

In [ ]:
class Squeeze_Momentum(AGENT):
  '''date 입력할때 2019-01-01, 2019-01-02 는 절대 입력 금지 / 스퀴즈모멘텀 공식땜에
  그냥 2019-01-15 이전 입력은 얘만 제외...귀찮아짐'''
  def __init__(self, Ratio_List: list=[], percentile: int=1, time_series: list = []):
    super().__init__(Ratio_List, percentile)
    self.time_series = time_series



  def buy(self, date: time.strftime("YY-mm-dd")):
    # 월급날
    self.payday(date)
    whole_money = self.money_df["Whole"]  # 아래에서 자꾸 바뀜

    if date.year == 2019:
      for i in range(len(self.Asset_Names)):
        # ratio 0 -> no buy
        if self.Ratio_List[i] == 0:
          continue
        if len(self.Data_List[i]) == 0:
          continue

        howmany = random.randrange(3, 11)  # 얼마나 종목 살건지 랜덤
        to_buy = []   # 살 df ||인덱스||
        buy_ratio = []  # 살 자산 각 상대 비율
        #* 자산 종류 안 /종목 종류
        for j in range(howmany):
          buy_ratio.append(random.random()*4)   # 다이나믹하게 *4 / 안살건 안사게
          to_buy.append(random.randrange(0, len(self.Data_List[i])))   # 다른것들은 최상단(순위권) 선호, 얘는 잡주까지 다
        buy_ratio = self.softmax(buy_ratio)  # norm

        usable_money = self.money_df["Whole"] * self.Ratio_List[i]  # money_df 자산 분위는 생성자에서 deter
        for j in range(howmany):
          to_spend_each = usable_money * buy_ratio[j]
          self.buy_each(i ,to_buy[j], to_spend_each, date)

      return



    # SM 계산 가능
    # n번 랜덤으로 종목 서치, 계산, 판단
    for i in range(len(self.Asset_Names)):
      # ratio 0 -> no buy
      if self.Ratio_List[i] == 0:
        continue
      if len(self.Data_List[i]) == 0:
        # print(0)
        continue

      usable_money = whole_money * self.Ratio_List[i]

      cnt = 0
      for trial in range(10):  # enter_long | short 만족하는 거 하나 나올때까지 랜덤 서치 / 10개 중 최대 2개
        if cnt >= 2:
          return
        to_buy_index = random.randrange(0, len(self.Data_List[i]))
        # df = self.Data_List[i][to_buy_index] #.loc[:date]
        SM = self.f_Squeeze_Momentum(self.Data_List[i][to_buy_index], date, self.time_series)
        # whats ur norm / 현재 기준 양수 만족 && 과거 5개 타임스탬프 안에 음수나 0 존재 && 이전 SM['value'] 보다는 큰 값이어야 함, 감소추세X & squeeze_off=True여야 &
        if SM == "buy":
          self.buy_each(i, to_buy_index, usable_money, date)
          cnt += 2
          return
        elif SM == "buy_half":
          self.buy_each(i, to_buy_index, usable_money/2, date)
          cnt += 1

      if cnt == 0:  # 아무것도 못삼 -> 저축을 해야하는데...
        # 일단 현재 자산이라도 돌아보기, 이것도 최대 10개만...
        return
        # self.buy_each()



  def sell(self, date: time.strftime("YY-mm-dd")):
    if date.year == 2019:
      prob = random.random()
      if prob > 0.9:
        for i in range(len(self.Asset_Names)):
          if self.Ratio_List[i] == 0:
            continue
          if len(self.Data_List[i]) == 0:
            continue
          if len(self.stock_list[i]) == 0:
            continue

          for j in range(random.randrange(0, len(self.stock_list[i]))):
            Index = random.randrange(0, len(self.stock_list[i]))
            Share = math.floor(random.random() * self.share_list[i][Index])
            Share = Share if Share != 0 else 1
            self.sell_each(i, Index, Share, date)
      return

    for i in range(len(self.Asset_Names)):
      if self.Ratio_List[i] == 0:
        continue
      if len(self.Data_List[i]) == 0:
        continue
      if len(self.stock_list[i]) == 0:
            continue

      # 모든 현 자산 SM 계산
      SI = 0  # Shifting Index
      for j in range(len(self.stock_list[i])):
        Index = j - SI
        data_index = self.stock_list[i][Index]
        SM = self.f_Squeeze_Momentum(self.Data_List[i][data_index], date, self.time_series)
        if SM == "sell":
          SI += 1
          Share = self.share_list[i][Index]
        elif SM == "sell_half":
          Share = self.share_list[i][Index] // 2
        else:
          continue
        if Share == 0:
          continue

        self.sell_each(i, Index, Share, date)




  def f_Squeeze_Momentum(self, df: pd.DataFrame(), date: time.strftime("YY-mm-dd"), time_series: list = []):
    # get stock prices / 이전 주가 없으면 버그남
    try:
      df = df.loc[time_series].loc[:date]
    except:
      return 'nothing'
    # df.columns.names = [None, None]
    # df = df.droplevel(level=1,axis=1)

    # parameter setup
    year_length = time_series[-1].year - time_series[0].year + 1
    k = int(len(time_series)/(12*year_length))  # 한달에 몇일
    length_li = [6, 11, 16, 21, 26]
    mult_li = [1.2, 1, 0.9, 0.8, 0.7]
    length = length_li[k-1]
    mult = 1.5 * mult_li[k-1]
    length_KC = length_li[k-1]
    mult_KC = 1.5

    # calculate BB
    m_avg = df['Close'].rolling(window=length).mean()
    m_std = df['Close'].rolling(window=length).std(ddof=0)
    df['upper_BB'] = m_avg + mult * m_std
    df['lower_BB'] = m_avg - mult * m_std

    # calculate True Range
    # https://alphasquare.co.kr/home/insight/posts/fcb90857-9485-44c5-9b95-49266c42410f
    # https://www.jaenung.net/tree/21667
    # https://medium.com/geekculture/implementing-the-most-popular-indicator-on-tradingview-using-python-239d579412ab
    df['tr0'] = abs(df["High"] - df["Low"])
    df['tr1'] = abs(df["High"] - df["Close"].shift())
    df['tr2'] = abs(df["Close"] - df["Low"].shift())
    df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)  #

    # calculate KC
    range_ma = df['tr'].rolling(window=length_KC).mean()
    df['upper_KC'] = m_avg + range_ma * mult_KC
    df['lower_KC'] = m_avg - range_ma * mult_KC

    # calculate bar value
    highest = df['High'].rolling(window = length_KC).max()
    lowest = df['Low'].rolling(window = length_KC).min()
    m1 = (highest + lowest)/2
    df['value'] = (df['Close'] - (m1 + m_avg)/2)
    fit_y = np.array(range(0,length_KC))
    df['value'] = df['value'].rolling(window = length_KC).apply(lambda x:
                              np.polyfit(fit_y, x, 1)[0] * (length_KC-1) +
                              np.polyfit(fit_y, x, 1)[1], raw=True)

    # check for 'squeeze'
    df['squeeze_on'] = (df['lower_BB'] > df['lower_KC']) & (df['upper_BB'] < df['upper_KC'])
    df['squeeze_off'] = (df['lower_BB'] < df['lower_KC']) & (df['upper_BB'] > df['upper_KC'])
    df['nosqz'] = (df['squeeze_off']==False) & (df['squeeze_on']==False)


    # NORM
    if df["squeeze_off"][-1] == False:
      return 'nothing' # meaningless moment

    # squeeze가 여기부턴 on
    # whats ur norm / 현재 기준 양수 or 음수 -> [ , ] && 과거 5개 타임스탬프 안에 음수나 0 존재 or 기울기 증가 추세 && 이전 SM['value'] 보다는 큰 값이어야 함, 감소추세X & squeeze_off=True여야 &

    # 일단 매수신호 포착
    if df["value"][-1] > 0:
      if df["value"][-1] < df["value"][-2]:  # 기울기 양수 & 감소추세
        if df["High"][-1] < df["High"][-2]:  # 가격 하락추세
          return "sell_half"  # 일단 판다? - 먹힌다고봄... / 다시사면댐 / 일부만팔기
        else:  # 가격이 일단 증가
          return "nothing"

      # 기울기 양수 & 증가추세 -> 기울기 추세 추적
      index = -2
      while df["value"][index] > 0:
        index -= 1

      # 기울기 양수 시작한지 얼마안댐 -> 무조건구매해 -> 1-2 month?
      if -index < k*2:
        return "buy"

      gradients = []
      for i in range(-index): # 방향: + -> -
        gradients.append(df["value"][-i-1] - df["value"][-i-2])
      prime_gradients = []
      for i in range(len(gradients)-1):  # 방향: - -> +
        prime_gradients.append(gradients[i] - gradients[i+1])

      # 기울기 증감 추세 확인 / 삼차함수 생각?
      cnt = 0
      if prime_gradients[-1] < 0: # 최소 매도 조건
        for i in range(1, len(prime_gradients)):
          if prime_gradients[-i-1] < prime_gradients[-i]:
            cnt += 1
      if cnt > k*2:
        return "sell_half"

      # 감소가 많아서 힘빠지는 경우
      cnt = 0
      for i in range(len(gradients)):
        if gradients[i] < 0:
          cnt += 1
      if cnt > k*2:
        return "sell_half"

      # 남은건: 어디가 고점인지 모르는 상황 || 들어가긴 늦은 상황 ...
      return "buy_half"




    # 매도신호 포착
    elif df['value'][-1] < 0:
      if df["value"][-1] > df["value"][-2]:  # 기울기 음수 & 상승추세
        if df["High"][-1] > df["High"][-2]:  # 가격 상승추세
          return "buy_half"  # 일단 buy
        else:  # 아직은 가격 감소, 존버 가치 유
          return "nothing"

      # 기울기 음수 & 감소추세 -> 기울기 추세 추적
      index = -2
      while df["value"][index] < 0:
        index -= 1

      # 기울기 음수 시작한지 얼마안댐 -> 무조건? 판매 / 음봉은 와리가리 심함, 나중에 다시 사면 대지 머
      if -index < k*2:
        return "sell"

      gradients = []
      for i in range(-index): # 방향: + -> -
        gradients.append(df["value"][-i-1] - df["value"][-i-2])
      prime_gradients = []
      for i in range(len(gradients)-1):  # 방향: - -> +
        prime_gradients.append(gradients[i] - gradients[i+1])

      # 기울기 증감 추세 확인 / 삼차함수 생각?
      cnt = 0
      if prime_gradients[-1] > 0: # 최소 매수 조건
        for i in range(1, len(prime_gradients)):
          if prime_gradients[-i-1] > prime_gradients[-i]:
            cnt += 1
      if cnt > k*2:
        return "buy_half"

      # 증가가 많아서 힘빠지는 경우
      cnt = 0
      for i in range(len(gradients)):
        if gradients[i] < 0:
          cnt += 1
      if cnt > k*2:
        return "buy_half"
      print()

      # 애매함
      return "sell_half"



    # 그냥 오르면 사기 / 떨어져도 0이니까 just sell
    elif df['value'] == 0:
      if df['close'][-1] > df['close'][-2]:
        return 'buy'
      else:
        return 'sell'
